In [1]:
"""
There is a list of most active Stocks on Yahoo Finance https://finance.yahoo.com/most-active.
You need to compose several sheets based on data about companies from this list.
To fetch data from webpage you can use requests lib. To parse html you can use beautiful soup lib or lxml.
Sheets which are needed:
1. 5 stocks with most youngest CEOs and print sheet to output. You can find CEO info in Profile tab of concrete stock.
    Sheet's fields: Name, Code, Country, Employees, CEO Name, CEO Year Born.
2. 10 stocks with best 52-Week Change. 52-Week Change placed on Statistics tab.
    Sheet's fields: Name, Code, 52-Week Change, Total Cash
3. 10 largest holds of Blackrock Inc. You can find related info on the Holders tab.
    Blackrock Inc is an investment management corporation.
    Sheet's fields: Name, Code, Shares, Date Reported, % Out, Value.
    All fields except first two should be taken from Holders tab.


Example for the first sheet (you need to use same sheet format):
==================================== 5 stocks with most youngest CEOs ===================================
| Name        | Code | Country       | Employees | CEO Name                             | CEO Year Born |
---------------------------------------------------------------------------------------------------------
| Pfizer Inc. | PFE  | United States | 78500     | Dr. Albert Bourla D.V.M., DVM, Ph.D. | 1962          |
...

About sheet format:
- sheet title should be aligned to center
- all columns should be aligned to the left
- empty line after sheet

Write at least 2 tests on your choose.
Links:
    - requests docs: https://docs.python-requests.org/en/latest/
    - beautiful soup docs: https://www.crummy.com/software/BeautifulSoup/bs4/doc/
    - lxml docs: https://lxml.de/
"""
import requests
from bs4 import BeautifulSoup as soup
import pandas as pd 
import bisect

def get_page(url):
    """Function to return html from specified url."""
    web_page = soup(requests.get(url, headers={'User-Agent': 'Custom'}).content, "html.parser")
    return web_page.find('body')

def get_active_stock_count():
    """Function to return total number of currently active stocks."""
    stock_page = get_page('https://finance.yahoo.com/most-active')
    stock_container = stock_page.select_one('#fin-scr-res-table div')
    stock_count_container = stock_container.select_one('div > span + span > span').text
    return int(stock_count_container.split('of')[1].replace('results', '').strip())

def generate_stock_symbols_lst():
    active_stock_count = get_active_stock_count() 
    body = get_page(f'https://finance.yahoo.com/most-active?offset=0&count={active_stock_count}') # Getting the page with all of the most active stocks
    quote_links = body.select('a[data-test="quoteLink"]') # Making a list of all the stock codes
    return [link.text for link in quote_links]

In [2]:
holders_page = get_page(f"https://finance.yahoo.com/quote/BLK/holders?p=BLK")
institutional_holders_table = holders_page.select_one('#Col1-1-Holders-Proxy > section > div + div > div + div + div > table > tbody')
mutual_holders_table = holders_page.select_one('#Col1-1-Holders-Proxy > section > div + div > div + div + div + div > table > tbody')

for m_holder in mutual_holders_table.select('tr'):
    m_value = int(m_holder.select_one('td:nth-last-child(1)').text.replace(",",""))
    m_perc_out = m_holder.select_one('td:nth-last-child(2)').text
    m_date_rep = m_holder.select_one('td:nth-last-child(3) > span').text
    m_shares = m_holder.select_one('td:nth-last-child(4)').text

for i_holder in institutional_holders_table.select('tr'):
    i_value = int(i_holder.select_one('td:nth-last-child(1)').text.replace(",",""))
    i_perc_out = i_holder.select_one('td:nth-last-child(2)').text
    i_date_rep = i_holder.select_one('td:nth-last-child(3) > span').text
    i_shares = i_holder.select_one('td:nth-last-child(4)').text

print(f"val: {i_value}, perc:{i_perc_out}, date:{i_date_rep}, shares:{i_shares}")


val: 1772112989, perc:1.77%, date:Dec 30, 2022, shares:2,652,705


In [3]:
"""Function to generate sheets."""
stock_symbols = generate_stock_symbols_lst()
required_fields = {"youngest_ceos":{"names": [], "symbols": [],"country":[], "employees":[], "ceo_names":[], "ceo_dobs":[]}, "52_week_change": {"names": [], "symbols": [], "52_change":[], "cash":[]}, "largest_holds":{"names": [], "symbols": [], "holders":[],"shares":[], "date_reported":[], "%_out":[], "values":[]}}

stock_5_youngest_lst = []
stats_10_best_change_lst = []
holders_10_largest_lst = []

print(required_fields)
for symbol in stock_symbols:
    # Selecting profile section and extracting the stock name, country and number of employees
    profile_page = get_page(f"https://finance.yahoo.com/quote/{symbol}/profile?p={symbol}")
    asset_profile = profile_page.select_one('div[data-test="asset-profile"] [data-test="qsp-profile"]')

    stock_name = asset_profile.select_one('h3').text.strip()
    stock_country = asset_profile.select_one('p a').previous_sibling.previous_sibling
    stock_num_employees = asset_profile.select_one('p:last-of-type span:last-of-type').text.replace(',', '')

    stats_page = get_page(f"https://finance.yahoo.com/quote/{symbol}/key-statistics?p={symbol}")
    stats_table = stats_page.select_one('[data-test="qsp-statistics"] :nth-child(2) > div + div table')
    stats_52_week_change = float(stats_table.select_one('tbody tr:nth-child(2) td:nth-child(2)').text.replace('%', ''))
    stats_cash = stats_page.select_one('#Col1-0-KeyStatistics-Proxy > section > div.Mstart\(a\).Mend\(a\) > div:nth-child(3) > div > div:nth-child(5) > div > div > table > tbody > tr.Bxz\(bb\).H\(36px\).BdY.Bdc\(\$seperatorColor\) > td.Fw\(500\).Ta\(end\).Pstart\(10px\).Miw\(60px\)').text

    # Getting the table with information about all the CEOs for the stock
    stock_ceo_table = profile_page.select_one('section.quote-subsection table tbody')

    if(len(stats_10_best_change_lst) < 10):
        bisect.insort(stats_10_best_change_lst, (-float(stats_52_week_change), stock_name, symbol, stats_cash))
    elif -stats_52_week_change < stats_10_best_change_lst[-1][0]:
        stats_10_best_change_lst.pop()
        bisect.insort(stats_10_best_change_lst, (-float(stats_52_week_change), stock_name, symbol, stats_cash))

    for stock_ceo in stock_ceo_table.select('tr'):
        stock_ceo_name = stock_ceo.select_one('td:first-child').text
        stock_ceo_dob = stock_ceo.select_one('td:nth-last-child(1)').text
        if stock_ceo_dob != "N/A":
            stock_ceo_dob = int(stock_ceo_dob)
            if len(stock_5_youngest_lst) < 5:
                bisect.insort(stock_5_youngest_lst, (-stock_ceo_dob, stock_name, symbol, stock_country, stock_num_employees, stock_ceo_name))
            elif -stock_ceo_dob < stock_5_youngest_lst[-1][0]:
                stock_5_youngest_lst.pop()
                bisect.insort(stock_5_youngest_lst, (-stock_ceo_dob, stock_name, symbol, stock_country, stock_num_employees, stock_ceo_name))

holders_page = get_page(f"https://finance.yahoo.com/quote/BLK/holders?p=BLK")
institutional_holders_table = holders_page.select_one('#Col1-1-Holders-Proxy > section > div + div > div + div + div > table > tbody')
mutual_holders_table = holders_page.select_one('#Col1-1-Holders-Proxy > section > div + div > div + div + div + div > table > tbody')

for m_holder in mutual_holders_table.select('tr'):
    m_value = m_holder.select_one('td:nth-last-child(1)').text
    m_perc_out = m_holder.select_one('td:nth-last-child(2)').text
    m_date_rep = m_holder.select_one('td:nth-last-child(3) > span').text
    m_shares = int(m_holder.select_one('td:nth-last-child(4)').text.replace(",",""))
    m_hold_name = m_holder.select_one('td:nth-last-child(5)').text
    
    if(len(holders_10_largest_lst) < 10):
        bisect.insort(holders_10_largest_lst, (-m_shares, 'BlackRock, Inc.', 'BLK', m_value, m_date_rep, m_perc_out, m_hold_name))
    elif -m_shares < holders_10_largest_lst[-1][0]:
        holders_10_largest_lst.pop()
        bisect.insort(holders_10_largest_lst, (-m_shares, 'BlackRock, Inc.', 'BLK', m_value, m_date_rep, m_perc_out, m_hold_name))

for i_holder in institutional_holders_table.select('tr'):
    i_value = i_holder.select_one('td:nth-last-child(1)').text
    i_perc_out = i_holder.select_one('td:nth-last-child(2)').text
    i_date_rep = i_holder.select_one('td:nth-last-child(3) > span').text
    i_shares = int(i_holder.select_one('td:nth-last-child(4)').text.replace(",",""))
    i_hold_name = i_holder.select_one('td:nth-last-child(5)').text

    if(len(holders_10_largest_lst) < 10):
        bisect.insort(holders_10_largest_lst, (-i_shares, 'BlackRock, Inc.', 'BLK', i_value, i_date_rep, i_perc_out, i_hold_name))
    elif -i_shares < holders_10_largest_lst[-1][0]:
        holders_10_largest_lst.pop()
        bisect.insort(holders_10_largest_lst, (-i_shares, 'BlackRock, Inc.', 'BLK', i_value, i_date_rep, i_perc_out, i_hold_name))
print(stats_10_best_change_lst)
for ceo in stock_5_youngest_lst:
    young_ceo_dob = -ceo[0]
    young_ceo_stk_name = ceo[1]
    young_ceo_symbol = ceo[2]
    young_ceo_country = ceo[3]
    young_ceo_num_empl = ceo[4]
    young_ceo_name = ceo[5]

    required_fields["youngest_ceos"]["names"].append(young_ceo_stk_name)
    required_fields["youngest_ceos"]["symbols"].append(young_ceo_symbol)

    required_fields["youngest_ceos"]["country"].append(young_ceo_country)
    required_fields["youngest_ceos"]["employees"].append(young_ceo_num_empl)
    required_fields["youngest_ceos"]["ceo_names"].append(young_ceo_name)
    required_fields["youngest_ceos"]["ceo_dobs"].append(young_ceo_dob)

for stat in stats_10_best_change_lst:
    stat_week_change = -stat[0]
    stat_stk_name = stat[1]
    stat_symbol = stat[2]
    stat_cash = stat[3]

    required_fields["52_week_change"]["names"].append(stat_stk_name)
    required_fields["52_week_change"]["symbols"].append(stat_symbol)

    required_fields["52_week_change"]["52_change"].append(stat_week_change)
    required_fields["52_week_change"]["cash"].append(stat_cash)

for hold in holders_10_largest_lst:
    hold_shares = -hold[0]
    hold_stk_name = hold[1]
    hold_symbol = hold[2]
    hold_val = hold[3]
    hold_date = hold[4]
    hold_perc = hold[5]
    holder_name = hold[6]

    required_fields["largest_holds"]["names"].append(hold_stk_name)
    required_fields["largest_holds"]["symbols"].append(hold_symbol)

    required_fields['largest_holds']['shares'].append(hold_shares)
    required_fields['largest_holds']['date_reported'].append(hold_date)
    required_fields['largest_holds']['%_out'].append(hold_perc)
    required_fields['largest_holds']['values'].append(hold_val)
    required_fields['largest_holds']['holders'].append(holder_name)
print(required_fields)

youngest_ceos_df = pd.DataFrame({'Name': required_fields["youngest_ceos"]["names"], 'Code': required_fields["youngest_ceos"]["symbols"], 'Country': required_fields["youngest_ceos"]["country"], 'Employees': required_fields["youngest_ceos"]["employees"], 'CEO Name': required_fields["youngest_ceos"]["ceo_names"], 'CEO Year Born':required_fields["youngest_ceos"]["ceo_dobs"]})
week_change_df = pd.DataFrame({'Name': required_fields["52_week_change"]["names"], 'Code': required_fields["52_week_change"]["symbols"], '52-Week Change': required_fields["52_week_change"]["52_change"], 'Total Cash': required_fields["52_week_change"]["cash"]})
largest_holders_df = pd.DataFrame({'Name': required_fields['largest_holds']['names'], 'Code': required_fields['largest_holds']['symbols'], 'Holders': required_fields["largest_holds"]["holders"], 'Shares': required_fields["largest_holds"]['shares'], 'Date Reported': required_fields['largest_holds']['date_reported'], '% Out': required_fields['largest_holds']['%_out'], 'Values': required_fields['largest_holds']['values']})

largest_holders_df.to_csv('largest_holders.csv')
largest_holders_df

{'youngest_ceos': {'names': [], 'symbols': [], 'country': [], 'employees': [], 'ceo_names': [], 'ceo_dobs': []}, '52_week_change': {'names': [], 'symbols': [], '52_change': [], 'cash': []}, 'largest_holds': {'names': [], 'symbols': [], 'holders': [], 'shares': [], 'date_reported': [], '%_out': [], 'values': []}}


In [ ]:
youngest_ceos_df.to_csv('youngest_ceos.csv')
youngest_ceos_df

,Name,Code,Country,Employees,CEO Name,CEO Year Born
0,"Coinbase Global, Inc.",COIN,United States,4510,Mr. Frederick Ernest Ehrsam III,1989
1,"SoFi Technologies, Inc.",SOFI,United States,4200,Ms. Lauren Stafford Webb,1986
2,"C3.ai, Inc.",AI,United States,704,Mr. Juho Parkkinen,1985
3,"SoFi Technologies, Inc.",SOFI,United States,4200,Mr. Christopher Lapointe,1985
4,"Tesla, Inc.",TSLA,United States,127855,Mr. Zachary John Planell Kirkhorn,1985


In [ ]:
week_change_df.to_csv('week_change.csv')
week_change_df

,Name,Code,52-Week Change,Total Cash
0,NVIDIA Corporation,NVDA,28.25,13.3B
1,"C3.ai, Inc.",AI,13.25,772.44M
2,Microsoft Corporation,MSFT,2.60,99.5B
3,"Advanced Micro Devices, Inc.",AMD,0.40,5.86B
4,Apple Inc.,AAPL,-3.36,51.36B
5,Itaú Unibanco Holding S.A.,ITUB,-12.34,414.1B
6,Alphabet Inc.,GOOGL,-16.66,113.76B
7,Ford Motor Company,F,-17.19,32.18B
8,"SoFi Technologies, Inc.",SOFI,-20.99,1.46B
9,Vale S.A.,VALE,-22.88,25.03B
